Imports

In [23]:
import cv2
import pandas as pd
import numpy as np
import mediapipe as mp
import os

In [24]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [25]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [26]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [27]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [16]:
include_50=pd.read_csv(r'D:/final year project/Train_Test_Split/train_include50.csv')
dataset_path_new = r"D:/final year project/dataset_include50/"
key_points_path=r"D:/final year project/sign-language-recognition/key_points/"
print(include_50)

    Category     Word         Video                       FilePath
0    Animals   4.Bird  MVI_2987.MOV   Animals/4. Bird/MVI_2987.MOV
1    Animals   4.Bird  MVI_3094.MOV   Animals/4. Bird/MVI_3094.MOV
2    Animals   4.Bird  MVI_3095.MOV   Animals/4. Bird/MVI_3095.MOV
3    Animals   4.Bird  MVI_8567.MP4   Animals/4. Bird/MVI_8567.MP4
4    Animals   4.Bird  MVI_3040.MOV   Animals/4. Bird/MVI_3040.MOV
..       ...      ...           ...                            ...
761  Society  2.Death  MVI_8663.MP4  Society/2. Death/MVI_8663.MP4
762  Society  2.Death  MVI_8930.MP4  Society/2. Death/MVI_8930.MP4
763  Society  2.Death  MVI_8664.MP4  Society/2. Death/MVI_8664.MP4
764  Society  2.Death  MVI_4802.MOV  Society/2. Death/MVI_4802.MOV
765  Society  2.Death  MVI_4257.MOV  Society/2. Death/MVI_4257.MOV

[766 rows x 4 columns]


In [11]:
for Category, Word,Video,FilePath in include_50.values:
    file_path=os.path.join(dataset_path_new,FilePath)#remove the filename and extension for folder creation
    print(file_path)
    cap=cv2.VideoCapture(file_path)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()


            # Make detections
            if ret:
                image, results = mediapipe_detection(frame, holistic)                
                # Draw landmarks
                draw_styled_landmarks(image, results)

                # Show to screen
                # cv2.imshow('OpenCV Feed', image)
                extract_keypoints_and_save(results,FilePath,key_points_path)
            else:
                break

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()


D:/final year project/dataset_include50/Animals/4. Bird/MVI_2987.MOV
D:/final year project/dataset_include50/Animals/4. Bird/MVI_3094.MOV
D:/final year project/dataset_include50/Animals/4. Bird/MVI_3095.MOV
D:/final year project/dataset_include50/Animals/4. Bird/MVI_8567.MP4
D:/final year project/dataset_include50/Animals/4. Bird/MVI_3040.MOV
D:/final year project/dataset_include50/Animals/4. Bird/MVI_3096.MOV
D:/final year project/dataset_include50/Animals/4. Bird/MVI_3037.MOV
D:/final year project/dataset_include50/Animals/4. Bird/MVI_3013.MOV
D:/final year project/dataset_include50/Animals/4. Bird/MVI_3012.MOV
D:/final year project/dataset_include50/Animals/4. Bird/MVI_3038.MOV
D:/final year project/dataset_include50/Animals/4. Bird/Extra/MVI_3045.MOV
D:/final year project/dataset_include50/Animals/4. Bird/Extra/MVI_3044.MOV
D:/final year project/dataset_include50/Animals/4. Bird/Extra/MVI_3043.MOV
D:/final year project/dataset_include50/Animals/4. Bird/Extra/MVI_3042.MOV
D:/final y

In [53]:
def extract_keypoints_and_save(results,file_path,key_points,frame_number):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    result=np.concatenate([pose, face, lh, rh])
    word_folder=os.path.join(key_points,file_path)[:-4]#remove the filename and extension for folder creation
    #before D:/final year project/dataset/Animals/4. Bird/MVI_2987.MOV
    #after D:/final year project/dataset/Animals/4. Bird
    #if folder doesn't exisit, create it
    if not os.path.exists(word_folder):
        print("creating folder:",word_folder)
        os.makedirs(word_folder)
    print(key_points,file_path[:-4]+'/'+str(frame_number)+'.npy')
    npy_path=os.path.join(key_points,file_path[:-4]+'/'+str(frame_number)+'.npy')
    np.save(npy_path,result)

Test Data save as key points

In [29]:
key_points_path_test=r"D:/final year project/sign-language-recognition/key_points_test/"
include_50_test=pd.read_csv(r'D:/final year project/Train_Test_Split/test_include50.csv')
dataset_path_new_test = r"D:/final year project/dataset_include50_test/"


In [55]:
for Category, Word,Video,FilePath in include_50_test.values:
    file_path=os.path.join(dataset_path_new_test,FilePath)#remove the filename and extension for folder creation
    print(file_path)
    cap=cv2.VideoCapture(file_path)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        frame_number=0
        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()

            # Make detections
            if ret:
                frame_number+=1
                image, results = mediapipe_detection(frame, holistic)                
                # Draw landmarks
                draw_styled_landmarks(image, results)

                # Show to screen
                # cv2.imshow('OpenCV Feed', image)
                extract_keypoints_and_save(results,FilePath,key_points_path_test,frame_number)
            else:
                frame_number=0
                break

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        frame_number=0
        cap.release()
        cv2.destroyAllWindows()

D:/final year project/dataset_include50_test/Animals/4. Bird/MVI_2988.MOV
D:/final year project/sign-language-recognition/key_points_test/ Animals/4. Bird/MVI_2988/1.npy
D:/final year project/sign-language-recognition/key_points_test/ Animals/4. Bird/MVI_2988/2.npy
D:/final year project/sign-language-recognition/key_points_test/ Animals/4. Bird/MVI_2988/3.npy
D:/final year project/sign-language-recognition/key_points_test/ Animals/4. Bird/MVI_2988/4.npy
D:/final year project/sign-language-recognition/key_points_test/ Animals/4. Bird/MVI_2988/5.npy
D:/final year project/sign-language-recognition/key_points_test/ Animals/4. Bird/MVI_2988/6.npy
D:/final year project/sign-language-recognition/key_points_test/ Animals/4. Bird/MVI_2988/7.npy
D:/final year project/sign-language-recognition/key_points_test/ Animals/4. Bird/MVI_2988/8.npy
D:/final year project/sign-language-recognition/key_points_test/ Animals/4. Bird/MVI_2988/9.npy
D:/final year project/sign-language-recognition/key_points_tes

In [10]:
import numpy as np

key_points=np.load(r"D:/final year project/sign-language-recognition/key_points/Adjectives/1. loud/MVI_5178.npy")

In [13]:
for key_point in key_points:
    print(key_point)

0.5065401792526245
0.16023075580596924
-0.5840834975242615
0.9996477961540222
0.5189928412437439
0.13563251495361328
-0.5632756948471069
0.9991671442985535
0.5248737931251526
0.13620810210704803
-0.5633965730667114
0.9992509484291077
0.5296324491500854
0.1373128741979599
-0.5635097622871399
0.9991508722305298
0.4979541003704071
0.1352267563343048
-0.5670225620269775
0.9989355802536011
0.49090301990509033
0.1360388845205307
-0.5671355724334717
0.998971700668335
0.4841228425502777
0.13723234832286835
-0.5673786997795105
0.9987905621528625
0.5382772088050842
0.15331891179084778
-0.3847021162509918
0.9988837242126465
0.47770336270332336
0.1519850194454193
-0.40287354588508606
0.998493492603302
0.519114077091217
0.19175532460212708
-0.5124662518501282
0.999748706817627
0.4924970865249634
0.19079269468784332
-0.5175876617431641
0.9996845126152039
0.5891015529632568
0.33002540469169617
-0.22029368579387665
0.9998378753662109
0.43117207288742065
0.31364479660987854
-0.2856547236442566
0.999575

In [19]:
for Category, Word,Video,FilePath in include_50.values:
    file_path=os.path.join(dataset_path_new,FilePath)#remove the filename and extension for folder creation
    print(file_path)
    cap=cv2.VideoCapture(file_path)
    count=0
    while cap.isOpened():
            # Read feed
            ret, frame = cap.read()

            

            # Make detections
            if ret:
                print(count)
                count+=1

            else:
                break

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()
    break
    


D:/final year project/dataset_include50/Animals/4. Bird/MVI_2987.MOV
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
